In [4]:
import os
import json
import openai
from dotenv import load_dotenv
import time

In [5]:
load_dotenv()

True

In [6]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [10]:
with open("./data/2019_11_KICE.json", "rb") as f:
    data = json.load(f)

In [11]:
total_score = 0
for pa in data:
    for problem in pa["problems"]:
        total_score += problem["score"]
assert (total_score == 100)
print("test passed")

test passed


In [12]:
for pa in data:
    for problem in pa["problems"]:
        print(
            f'질문 : {problem["question"]}\n1번 : {problem["choices"][0]}\n2번 : {problem["choices"][1]}\n3번 : {problem["choices"][2]}\n4번 : {problem["choices"][3]}\n5번 : {problem["choices"][4]}\n정답 : {problem["answer"]}\n점수 : {problem["score"]}\n\n')

질문 :  위 방송 진행자의 말하기 방식에 대한 설명으로 가장 적절한 것은?
1번 : 사연 내용을 정리하고 사연 신청자의 마음에 공감하고 있다.
2번 : 사연 신청자의 궁금증을 해소하고 다음 방송을 예고하고 있다.
3번 : 사연 내용을 선정하게 된 동기를 밝히고 청취자의 참여를 독려하고 있다.
4번 : 사연과 관련된 자신의 과거 경력을 소개하고 전문성을 부각하고 있다.
5번 : 사연에 대한 상담 중에 질문을 던지고 사연 속 상황을 다양한 관점에서 생각해 보도록 유도하고 있다.
정답 : 1
점수 : 2


질문 :  다음은 위 방송을 진행하기 위해 진행자가 세운 계획이다. 방송에 반영되지 않은 것은?
1번 : ㉠
2번 : ㉡
3번 : ㉢
4번 : ㉣
5번 : ㉤
정답 : 3
점수 : 2


질문 :  다음은 위 방송을 들은 청취자들이 게시판에 올린 댓글이다. 방송 내용을 고려하여 청취자들의 반응을 분석한 것으로 적절하지 않은 것은? [3점]
1번 : ‘청취자 1’은 자신이 방송을 들은 목적과 관련해 방송 내용이 충분하지 않다고 판단하고 있군.
2번 : ‘청취자 2’는 방송 내용을 이해한 바를 확인하고 방송에서 안내되지 않았던 방법의 효과를 예측하고 있군.
3번 : ‘청취자 3’은 방송에서 언급한 방법을 다른 사람들에게 권유하고 적용할 것을 다짐하고 있군.
4번 : ‘청취자 4’는 방송에서 제시한 방법을 다른 경우에도 적용할 수 있는지 궁금해하고 있군.
5번 : ‘청취자 5’는 방송에서 언급한 방법을 긍정적으로 평가하고 자신의 언어 습관을 반성하고 있군.
정답 : 3
점수 : 3


질문 : ‘학생 3’이 (나)를 참고하여 (가)를 고쳐 쓰기 위해 세운 계획으로 적절하지 않은 것은?
1번 : ㉮
2번 : ㉯
3번 : ㉰
4번 : ㉱
5번 : ㉲
정답 : 4
점수 : 2


질문 : (나)를 바탕으로 할 때, (가)의 마지막 부분에 추가로 작성할 내용으로 가장 적절한 것은?
1번 : 학생회장은 “행사 홍보가 힘들었지만 즐거운 경험이었다.”라고 밝혔다. 선생님과 학생

# Test

In [13]:
def get_answer_basic(paragraph, question, choices, question_plus="", get_prompt=False, no_paragraph=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 수험생으로써 다음 문제의 답을 구하세요.
    """
    if not no_paragraph:
        user_prompt = f"""
            지문 :
            {paragraph}
        """
    else:
        user_prompt = ""
    if question_plus:
        user_prompt += f"""
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        1번, 2번, 3번, 4번, 5번 중 정답을 말하세요. 풀이는 말하지 마세요.
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [13]:
def get_answer(paragraph, question, choices, question_plus="", get_prompt=False):
    system_prompt = """
        국어 시험 문제를 푸는 천재 수험생으로써 다음 문제의 답을 구하세요.
        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.
        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.
        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.
        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.
        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.
        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.

        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 '단서'와 추론에 필요한 핵심적인 정보를 연결지어 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.

        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.

        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.

        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표시한 것입니다. 지문에서 [A]부분을 찾고, 그 이후에 소괄호 () 안에 들어있는 내용들을 참고하세요.
        ㉠~㉤ 등의 표시에 경우에는 특정 단어를 지문에서 표시한 것입니다. 지문에서 각 부분을 찾고, 이후에 소괄호 ()안에 들어있는 단어를 참고하세요.

        또한, (가) 혹은 (나) 글 등으로 언급이 되어있는 것은, (가) 이후부터 (나) 이전까지, 역시 (나) 이후부터 끝까지, 혹은 (다) 이전까지의 글을 의미한 것이므로, 문제를 풀이할 때에 해당 부분의 글들을 참고하여 문제를 풀이하세요.
    """
    user_prompt = f"""
        지문 :
        {paragraph}
    """
    if question_plus:
        user_prompt += f"""
            이 문제는 아래와 같이 <보기>가 주어져 있습니다. 문제의 각 선택지들을 해결하기 위한 배경 지식을 설명해 주고 있는 것이 <보기>로써, 각 선택지들을 지문과 연결시키고, <보기>의 지식을 활용하면 각 선택지의 참과 거짓을 판단할 수 있습니다.
            문제를 해결할 때, 반드시 <보기>의 내용을 이용해서 문제를 해결해야 합니다.
            <보기> :
            {question_plus}
        """
    user_prompt += f"""
        질문 :
        {question}

        선택지 :
        1번 - {choices[0]}
        2번 - {choices[1]}
        3번 - {choices[2]}
        4번 - {choices[3]}
        5번 - {choices[4]}

        1번, 2번, 3번, 4번, 5번 각각에 대한 풀이와 최종 정답을 서술하세요. <보기>가 있다면, <보기>를 적극적으로 활용하여 최종 정답을 구하세요.
        선택지 중에 정답이 없는 경우는 없으니, 정답이 없는 것 같다면 기존에 검토했던 지문이 아닌 다른 지문을 다시 검토하여 구해보세요. 예를 들어, (나) 지문만을 사용했다면 (가) 지문을 검토하고, (가) 지문만을 해답에 사용했다면 (나) 지문을 해답에 사용하여 해답을 구해보세요. 이 때, 반드시 모든 선택지들을 다시 검토하세요.
    """
    if get_prompt:
        return system_prompt +"\n\n" +user_prompt

    completion = openai.ChatCompletion.create(model="gpt-4", messages=[{
        "role": "system", "content": system_prompt
    }, {
        "role": "user", "content": user_prompt
    }], temperature=0.0)
    return completion.choices[0].message.content

In [15]:
def get_answer_one_problem(paragraph_num: int, problem_num: int, get_prompt=False):
    problem = data[paragraph_num]["problems"][problem_num]
    no_paragraph = False
    if "no_paragraph" in list(problem.keys()):
        no_paragraph = True
    if "question_plus" in list(problem.keys()):
        question_plus_text = problem["question_plus"]
    else:
        question_plus_text = ""
    return get_answer_basic(paragraph=data[paragraph_num]["paragraph"],
                      question=problem["question"],
                      choices=problem["choices"],
                      question_plus=question_plus_text,
                      get_prompt=get_prompt,
                      no_paragraph=no_paragraph)

In [19]:
from tqdm.notebook import tqdm
import time
answers_list = list()
for paragraph_index, paragraph in enumerate(data):
    for problem_index, problem in tqdm(enumerate(paragraph["problems"])):
        answers_list.append(get_answer_one_problem(paragraph_index, problem_index))
        time.sleep(20)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [22]:
answers_list

['1번',
 '3번',
 '3번',
 '4번',
 '2번',
 '5번',
 '3번',
 '2번',
 '2번',
 '2번',
 '4번',
 '2번',
 '4번',
 '4번',
 '정답은 1번입니다.',
 '3번',
 '5번',
 '1번',
 '3번',
 '1번',
 '4번',
 '1번',
 '4번',
 '4번',
 '5번',
 '5번',
 '2번',
 '5번',
 '3번',
 '2번',
 '1번',
 '2번',
 '1번',
 '4번',
 '3번',
 '5번',
 '5번',
 '4번',
 '1번, 5번',
 '4번',
 '3번',
 '4번',
 '1번',
 '3번',
 '4번']

In [16]:
get_answer_one_problem(0,0)

'1번'

In [9]:
#21번 문제 - 정답 5번
get_answer_one_problem(4,3, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 푸는 천재 수험생으로써 다음 문제의 답을 구하세요.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'단서\'와 추론에 필요한 핵심적인 정보를 연결지어 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표시한 것입니다. 지문에서 [A]부분을 찾고, 그 이후에 소괄호 () 안에 들어있는 내용들을 참고하세요.     

In [31]:
# 29번 문제 - 정답 1번
get_answer_one_problem(6,2, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 푸는 똑똑한 학생으로써 다음 문제의 답을 구하세요.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표시한 것입니다. 지문에서 [A]부분을 찾고, 그 이후에 소괄호 () 안에 들어있는 내용들을 참고하세요.        ㉠~㉤ 등의 표시에

In [34]:
# 31번 문제 - 정답 1번
get_answer_one_problem(7,0, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 풀이하는 똑똑한 한국어 선생님으로써 다음 문제의 답을 구하세요. 똑똑한 한국어 선생님은 기존의 한국어 지식, 문법, 어휘, 글의 구조, 풀이 방법 등을 이용하여 답을 구할 수 있습니다.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표

In [35]:
# 40번 문제 - 정답 3번
get_answer_one_problem(9,2, get_prompt=True).replace("\n", "")

'        국어 시험 문제를 풀이하는 똑똑한 한국어 선생님으로써 다음 문제의 답을 구하세요. 똑똑한 한국어 선생님은 기존의 한국어 지식, 문법, 어휘, 글의 구조, 풀이 방법 등을 이용하여 답을 구할 수 있습니다.        답을 구하는 방법은 다음과 같습니다. 해당 문제에서 5개의 선택지가 주어집니다.        각각의 선택지는 각각 참 (예)이거나 거짓 (아니오)입니다. 애매하거나 답을 구할 수 없는 선택지는 존재하지 않습니다.        1번부터 5번까지 각 선택지에 대한 참과 거짓을 판단하세요.        문제의 질문에 따라 유일한 참인 선택지를 선택하거나, 유일한 거짓인 선택지를 선택해야 합니다.        문제의 질문이 "가장 적절한 것은?"으로 끝나면 유일한 참인 선택지를 선택하세요.        반대로, 문제의 질문이 "적절하지 않은 것은?"으로 끝나면 유일한 거짓인 선택지를 선택하세요.        문제를 풀기 위하여, 먼저 주어진 지문의 정보를 참고하여 각 선택지에 대해서 참과 거짓을 구별하세요. 구별할 때에는 지문에 제시되어 있는 \'증거\'와 그 추론 과정을 간단히 설명하세요. 다만, 단순하게 지문의 내용의 사실 관계 만으로 1차원적인 정답을 구하지 않고, 실제 국어 문제를 풀이하는 선생님과 같은 관점에서 적절한 정답을 찾아가야 합니다.        이후, 최종적으로 유일한 참 혹은 유일한 거짓인 선택지를 고릅니다. 마지막으로, 적절한 답안은 "1번"과 같이 선택지의 번호만을 적으면 됩니다.        만약 유일한 참 혹은 유일한 거짓이 없다면, 적합한 사고 과정을 거치지 않거나 충분하게 지문을 참조하지 않은 것입니다. 하나의 선택지에 참고해야할 부분이 지문에서 여러 개가 있을 수 있으며, (가) 및 (나) 등을 비롯한 여러 글에서 참고해야할 부분이 있을 수 있습니다. 이러한 부분들을 반드시 모두 참고하여 문제를 풀이해야 합니다.        또한, 문제에서 [A]나 [B]를 언급한 경우, 이는 지문 안의 여러 문장들을 표

In [16]:
# 41번 문제 - 정답 2번
get_answer_one_problem(9,3, get_prompt=False).replace("\n", "")

'1번 선택지는 (나) 지문에서 "학생 2 : 근데 발표회까지 얼마 안 남아서 국가 보호종을 비롯해 주목할 만한 몇몇 식물만 표시해야 할 듯해. 그리고 식물 이름은 ○○ 고등학교처럼 누리집을 이용해 편리하게 찾자."라는 부분에서 확인할 수 있습니다. 따라서 1번은 참입니다.2번 선택지는 (나) 지문에서 "학생 2 : 근데 발표회까지 얼마 안 남아서 국가 보호종을 비롯해 주목할 만한 몇몇 식물만 표시해야 할 듯해."라는 부분에서 확인할 수 있습니다. 따라서 2번은 참입니다.3번 선택지는 (나) 지문에서 "학생 2 : 생각해 봤는데, 우리 셋이서 ○○ 고등학교가 한 것처럼 넓은 공간을 조사하긴 힘들 듯하니 학교에서 걸어갈 만한 거리만 지도의 범위로 삼는 게 좋지 않을까?"라는 부분에서 확인할 수 있습니다. 따라서 3번은 참입니다.4번 선택지는 (나) 지문에서 "학생 2 : 여러 종류의 식물이 있는 곳도 좋지만, 나는 우리 학교 학생들이 볼 지도이니 학생들에게 친숙한 장소가 더 좋을 듯해."라는 부분에서 확인할 수 있습니다. 따라서 4번은 참입니다.5번 선택지는 (나) 지문에서 "학생 2 : 그래. 그런데 ○○ 고등학교가 이어 붙이는 방식으로 지도를 만든 건 참신하긴 한데 통일감이 없어 부자연스러울 듯해. 우리는 조사한 내용을 모아 함께 지도를 그리자."라는 부분에서 확인할 수 있습니다. 따라서 5번은 참입니다.따라서, 모든 선택지가 참인 것으로 판단되므로, 문제에서 요구하는 \'적절하지 않은 것\'을 찾지 못했습니다. 이 경우, 지문을 다시 검토하거나, 다른 관점에서 선택지를 검토해야 합니다.'

In [15]:
get_answer_one_problem(0,1, get_prompt=False).replace("\n", "")

'1번 선택지는 ⓐ와 ⓑ에 모두 ‘독자 자신에게서 답을 찾는 질문’이 나타난다고 했는데, ⓐ는 \'음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?\'라는 질문으로, 이는 책에서 답을 찾는 질문이다. ⓑ는 \'이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.\'라는 책의 내용을 인용한 부분으로, 이 역시 책에서 답을 찾는 질문이다. 따라서 1번은 거짓이다.2번 선택지는 ⓒ와 ⓓ에 모두 ‘책에 명시된 내용’에서 질문의 답을 찾아내는 모습이 나타난다고 했는데, ⓒ는 \'절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.\'라는 책의 내용을 종합하여 질문의 답을 찾아낸 부분이다. 하지만 ⓓ는 \'내가 진정으로 좋아하는 일이 무엇인지 나에게 묻게 되었다.\'라는 부분으로, 이는 독자 자신에게서 답을 찾는 질문이다. 따라서 2번은 거짓이다.3번 선택지는 ⓐ에는 ‘책에서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다고 했는데, ⓐ는 \'음악 시간에 들었던 베토벤의 교향곡 <합창>이 위대한 작품인 이유는 무엇일까?\'라는 질문으로, 이는 책에서 답을 찾는 질문이다. ⓔ는 \'글 쓰는 일에서 가장 큰 행복을 느꼈던 나를 발견\'할 수 있었다는 부분으로, 이는 독자의 경험에서 답을 찾아낸 부분이다. 따라서 3번은 참이다.4번 선택지는 ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다고 했는데, ⓑ는 \'이 곡이 낭만주의 음악의 특징을 보여 준다고 했다.\'라는 책의 내용을 인용한 부분으로, 이는 책에서 답을 찾는 질문이다. ⓒ는 \'절망적 상황에서도 열정적으로 자신이 좋아하는 일을 했기에 교향곡 구성의 새로움을 보여 준 명작이 탄생했음을 알게 됐다.\'라는 책의 내용을 종합하여 질문의 답을 찾아낸 부분이다. 따라서 4번은 참이다.5번 선택지는 ⓓ에는 ‘독자 자신에게서 답을

In [ ]:
answers = list()
answer_num = 1
for pa in data:
    for problem in pa["problems"]:
        if "question_plus" in list(problem.keys()):
            question_plus_text = problem["question_plus"]
        else:
            question_plus_text = ""
        out = get_answer(paragraph=pa["paragraph"],
                         question=problem["question"],
                         choices=problem["choices"],
                         question_plus=question_plus_text)
        print(f'{answer_num}번 문제 : 답 - \n{out}\n')
        answer_num += 1
        answers.append(out)
        time.sleep(15)

1번 문제 : 답 - 
1번 선택지는 "[A] 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다." 에서 확인할 수 있습니다. 따라서 1번은 참입니다.

2번 선택지는 "한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.)" 그리고 "동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다." 에서 독자가 다른 독자와의 소통을 통해 인식의 폭을 확장할 수 있다는 내용을 확인할 수 있습니다. 따라서 2번은 참입니다.

3번 선택지는 "독자는 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다." 에서 확인할 수 있습니다. 독자는 책을 통해 필자를 매개로 직접 경험해 보지 못했던 다양한 삶에 접할 수 있습니다. 따라서 3번은 참입니다.

4번 선택지는 "[A] 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다."에서 확인할 수 있습니다. 따라서 4번은 참입니다.

5번 선택지는 "독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다."라는 문장에서, 독자가 자신이 속한 사회나 시대의 영향을 받으며 필자와 소통한다는 내용을 포함하고 있습니다. 하지만 이 문장이 의미하는 바는 필자의 사회나 시대를 경험하며 필자와 소통한다는 것이지, 독자가 자신이 속한 사회나 시대의 영향을 받으며 필자와 대화한다는 내용은 없습니다. 따라서 5번은 거짓입니다.

따라서 최종적으로 지문 내용과 일치하지 않는 것은 "5번"입니다.

2번 문제 : 답 -

In [23]:
solutions = list()
for pa in data:
    for problem in pa["problems"]:
        solutions.append(problem["answer"])

In [24]:
scores = list()
for pa in data:
    for problem in pa["problems"]:
        scores.append(problem["score"])

In [29]:
f = open("./result/2019_11_KICE-gpt4_baseline.txt", 'w')
for i, item in enumerate(answers_list):
    txt = f'{i+1}번 문제 : {item}\n정답 : {solutions[i]}\n배점 : {scores[i]}\n----------------------------\n'
    print(txt)
    f.write(txt)
f.close()

1번 문제 : 1번
정답 : 1
배점 : 2
----------------------------

2번 문제 : 3번
정답 : 3
배점 : 2
----------------------------

3번 문제 : 3번
정답 : 3
배점 : 3
----------------------------

4번 문제 : 4번
정답 : 4
배점 : 2
----------------------------

5번 문제 : 2번
정답 : 4
배점 : 2
----------------------------

6번 문제 : 5번
정답 : 5
배점 : 2
----------------------------

7번 문제 : 3번
정답 : 3
배점 : 2
----------------------------

8번 문제 : 2번
정답 : 2
배점 : 2
----------------------------

9번 문제 : 2번
정답 : 2
배점 : 2
----------------------------

10번 문제 : 2번
정답 : 2
배점 : 3
----------------------------

11번 문제 : 4번
정답 : 3
배점 : 2
----------------------------

12번 문제 : 2번
정답 : 2
배점 : 2
----------------------------

13번 문제 : 4번
정답 : 5
배점 : 3
----------------------------

14번 문제 : 4번
정답 : 3
배점 : 2
----------------------------

15번 문제 : 정답은 1번입니다.
정답 : 5
배점 : 2
----------------------------

16번 문제 : 3번
정답 : 3
배점 : 2
----------------------------

17번 문제 : 5번
정답 : 5
배점 : 2
----------------------------

18번 문제 : 1번
정답 : 1
배점 : 2
-----------------------